In [ ]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = data["text"]
labels_raw = data["label_annotated"]
labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, 
    labels, 
    test_size=0.2, 
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 120
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 30
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=10)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [14]:
trainer.train()

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 3.6659796237945557, 'eval_accuracy': 0.6, 'eval_runtime': 0.3064, 'eval_samples_per_second': 97.907, 'eval_steps_per_second': 13.054, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.923147678375244, 'eval_accuracy': 0.7, 'eval_runtime': 0.2904, 'eval_samples_per_second': 103.302, 'eval_steps_per_second': 13.774, 'epoch': 2.0}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.7036949396133423, 'eval_accuracy': 0.8333333333333334, 'eval_runtime': 0.2945, 'eval_samples_per_second': 101.87, 'eval_steps_per_second': 13.583, 'epoch': 9.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.7759512662887573, 'eval_accuracy': 0.8333333333333334, 'eval_runtime': 0.2705, 'eval_samples_per_second': 110.91, 'eval_steps_per_second': 14.788, 'epoch': 10.0}
{'train_runtime': 38.6935, 'train_samples_per_second': 31.013, 'train_steps_per_second': 3.877, 'train_loss': 0.0534316889444987, 'epoch': 10.0}


TrainOutput(global_step=150, training_loss=0.0534316889444987, metrics={'train_runtime': 38.6935, 'train_samples_per_second': 31.013, 'train_steps_per_second': 3.877, 'total_flos': 315733266432000.0, 'train_loss': 0.0534316889444987, 'epoch': 10.0})